# What is Selenium?

## 1

What environment do we use?

See [What is Docker?](What%20is%20Docker.ipynb)

Run Selenium in Docker?


In [1]:
var path = require('path');
var corePath = path.resolve(path.join(__dirname, '..', 'Core'));
var importer = require(corePath);
var writeFileSync = require('fs').writeFileSync;

// add some run commands to the bash script
var bashToRun = (code) => code.split('\n').map(l => 'RUN ' + l)
    .join('\n').replace(/\\\s*\nRUN\s*/ig, '\\\n ');

// create a selenium Dockerfile with a vnc connection
var interpretAllPath = path.join('..', 'Core', 'How to interpret all notebooks.ipynb');
var interpret, interpretObject;
var seleniumDocker = (outputFile) => {
    return importer.importNotebook(interpretAllPath)
        .then((f) => {
            interpret = f['interpret'];
            interpretObject = f['interpretObject'];
        })
        .then(() => Promise.all([
            interpret('run selenium'),
            // add some extra services
            interpret('linux dev tools'),
            interpret('vnc html'),
            interpret('vnc docker')
        ]))
        .then(r => r.map(r => r[0]))
        .then(r => interpretObject(r))
        .then(r => {
            // lets fix the middle result
            r[1].code = bashToRun(r[1].code);
            r[2].code = bashToRun(r[2].code);
            return r.map(r => r.code).join('\n');
        })
        .then(r => {
            // save the Dockerfile
            writeFileSync(outputFile, r);
            return r;
        });
};
(seleniumDocker);


[Function: seleniumDocker]

How to set up selenium server?

In [7]:
var path = require('path');
var execSync = require('child_process').execSync;
var DOWNLOAD_DIR = path.join(process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE, 'Downloads');
var PROFILE_DIR = path.join(process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE, '.defaultProfile');
var getSeleniumServer = () => {
    try {
        fs.mkdirSync(DOWNLOAD_DIR);
    } catch (err) {
        if (err.code != 'EEXIST') {
            throw err;
        }
    }
    try {
        fs.mkdirSync(PROFILE_DIR);
    } catch (err) {
        if (err.code != 'EEXIST') {
            throw err;
        }
    }
    try {
        fs.unlinkSync(path.join(PROFILE_DIR, 'SingletonLock'));
    } catch (err) {
        if (err.code != 'ENOENT') {
            throw err;
        }
    }
    return seleniumDocker(path.join(__dirname, 'Dockerfile'))
        .then(r => execSync('docker build -t act-selenium .').toString())
        .then(r => {
            var ps = execSync('docker ps -a').toString();
            if(ps.indexOf('act-selenium') !== -1) {
                return r 
                    + execSync('docker stop act-selenium').toString()
                    + execSync('docker rm act-selenium').toString();
            }
            return '';
        })
        .then(r => r + execSync('docker run --shm-size=2g --name act-selenium -d -p 8888:8888 -p 6080:6080 -p 5900:5900 -p 4444:4444 -p 4200:4200 -p 3000:3000 -v "' + DOWNLOAD_DIR + '":/data/downloads act-selenium').toString())
        // wait for process to start
        .then(() => new Promise((resolve, reject) =>
                                setTimeout(resolve, 5000)));
};
(getSeleniumServer);


[Function: getSeleniumServer]

In [8]:
$$.async();
getSeleniumServer()
    .then(r => $$.sendResult(r))
    .catch(r => $$.sendError(r));

Already imported
[ 'How to import a notebook.ipynb',
  'How to interpret all notebooks.ipynb' ]


Error: ENOENT: no such file or directory, unlink '/Users/briancullinan/.defaultProfile/SingletonLock'
    at Error (native)

# 2

How to connect to VNC through the web browser?

[Click here to open](http://localhost:6080/vnc.html?password=secret&host=localhost&port=6080&autoconnect=true&resize=downscale&view_only=true&reconnect=true) in a separate browser


In [6]:
var vncIframe = () => `
<div style="display:block; width:100%; padding-bottom:75%;position:relative;">
<iframe id="vnc"
style="position:absolute;top:0;right:0;bottom:0;left:0;width:100%;height:100%;border:0;" 
src="http://localhost:6080/vnc.html?password=secret&amp;host=localhost&amp;port=6080&amp;autoconnect=true&amp;resize=downscale&amp;view_only=true&amp;reconnect=true"></iframe></div>
`;
(vncIframe);

In [8]:
$$.mime({'text/html': vncIframe()});

## 3

How to run a selenium test on the Docker machine?
